In [1]:
!pip install tensorflow-gpu tensorflow


Defaulting to user installation because normal site-packages is not writeable


reference from https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')



In [3]:
import tensorflow as tf
print(tf.__version__)

2.8.0


# cloning tfod from github

In [4]:
cd /home/ovenoboyo/projects/pbl/training/

/home/ovenoboyo/projects/pbl/training


In [5]:
# !git clone https://github.com/tensorflow/models.git './cloned_tfod2'

In [6]:
pwd

'/home/ovenoboyo/projects/pbl/training'

In [7]:
cd /home/ovenoboyo/projects/pbl/training/cloned_tfod2/research

/home/ovenoboyo/projects/pbl/training/cloned_tfod2/research


In [8]:
# !protoc object_detection/protos/*.proto --python_out=.

In [9]:
# !git clone https://github.com/cocodataset/cocoapi.git

In [10]:
cd cocoapi/PythonAPI

/home/ovenoboyo/projects/pbl/training/cloned_tfod2/research/cocoapi/PythonAPI


In [11]:
# !make

In [12]:
# cp -r pycocotools /home/ovenoboyo/projects/pbl/training/cloned_tfod2/research

# installing obj-detection API

In [13]:
cd ..

/home/ovenoboyo/projects/pbl/training/cloned_tfod2/research/cocoapi


In [14]:
cd ..

/home/ovenoboyo/projects/pbl/training/cloned_tfod2/research


In [15]:
cp object_detection/packages/tf2/setup.py .

In [16]:
!python -m pip install .

Defaulting to user installation because normal site-packages is not writeable
Processing /home/ovenoboyo/projects/pbl/training/cloned_tfod2/research


  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1657764 sha256=8ae8663b69f15460dca5ec6c1963b0059b15cb4df186f6c5c2371e7640bda33e
  Stored in directory: /tmp/pip-ephem-wheel-cache-v9fkok36/wheels/16/24/ac/4656166192587b8b9c043268262805ca1fe2dfe25ce38ecfef
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1


In [17]:
pwd

'/home/ovenoboyo/projects/pbl/training/cloned_tfod2/research'

In [18]:
# From within TensorFlow/models/research/
# Passes successfully
# !python object_detection/builders/model_builder_tf2_test.py 

# pre trained models


In [19]:
cd /home/ovenoboyo/projects/pbl/training/pretrained_models

/home/ovenoboyo/projects/pbl/training/pretrained_models


In [20]:
# downloading models from https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
# !wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz 

In [21]:
# !tar xvf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

In [22]:
pwd

'/home/ovenoboyo/projects/pbl/training/pretrained_models'

# Creating tf records and labelmap

In [23]:
labels = [{'name':'fork', 'id':1}, {'name':'gun', 'id':2}, {'name':'knife', 'id':3}, {'name':'screwdriver', 'id':4}, {'name':'wrench', 'id':5}]

In [24]:
labels

[{'name': 'fork', 'id': 1},
 {'name': 'gun', 'id': 2},
 {'name': 'knife', 'id': 3},
 {'name': 'screwdriver', 'id': 4},
 {'name': 'wrench', 'id': 5}]

In [25]:
with open(r'/home/ovenoboyo/projects/pbl/training/annotations'+'/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n') 
        f.write('\tname:\'{}\'\n'.format(label['name'])) #writing item name in pbtxt file
        f.write('\tid:{}\n'.format(label['id'])) #adding the id for item in pbtxt file
        f.write('}\n')
#pbtxt are binary files, reqd by tf for making a label map in object detection 

In [26]:
# reference from cod1.ipynb section creating tf records

In [7]:
cd /home/ovenoboyo/projects/pbl/training/scripts

/home/ovenoboyo/projects/pbl/training/scripts


In [8]:
#train record
!python generate_tfrecord.py -x /home/ovenoboyo/projects/pbl/training/images/train -l /home/ovenoboyo/projects/pbl/training/annotations/label_map.pbtxt -o /home/ovenoboyo/projects/pbl/training/annotations/train.record


Successfully created the TFRecord file: /home/ovenoboyo/projects/pbl/training/annotations/train.record


In [9]:
#test record
!python generate_tfrecord.py -x /home/ovenoboyo/projects/pbl/training/images/test -l /home/ovenoboyo/projects/pbl/training/annotations/label_map.pbtxt -o /home/ovenoboyo/projects/pbl/training/annotations/test.record


Successfully created the TFRecord file: /home/ovenoboyo/projects/pbl/training/annotations/test.record


altering pipeline.config from model folder

In [30]:
# changes made are:
# num_classes 
# and batch_size(8) line 131
# line 161 fine tune checkpoint : path to ckpt-0.index
# line 167 fine_tune_checkpoint_type: "detection"
# use_bfloat16: false
# line 172 in train_input_reader { paths of label map and train record}
# in eval_input_reader { paths of label map and test record}
# lines 152 and 162 - num of steps

In [31]:
pwd

'/home/ovenoboyo/projects/pbl/training/scripts'

In [32]:
cd /home/ovenoboyo/projects/pbl/training

/home/ovenoboyo/projects/pbl/training


In [33]:
ls

annotations/                images/             saved_model/
cloned_tfod2/               model_main_tf2.py   scripts/
Copy_of_pbl2a.ipynb         models/             training_scripts.zip
exporter_main_v2.py         pretrained_models/
export_tflite_graph_tf2.py  result.png


In [34]:
# cd '/content/drive/MyDrive/ObjDec'

In [35]:
# ls

# training

In [ ]:
!TF_GPU_ALLOCATOR=cuda_malloc_async python /home/ovenoboyo/projects/pbl/training/model_main_tf2.py --model_dir='/home/ovenoboyo/projects/pbl/training/models/my_ssdresnet_101/' --pipeline_config_path='/home/ovenoboyo/projects/pbl/training/models/my_ssdresnet_101/pipeline.config'

2022-04-26 12:20:50.936630: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 12:20:50.983273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 12:20:50.983463: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 12:20:50.983847: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

/home/ovenoboyo/.local/lib/python3.8/site-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '
2022-04-26 12:21:18.606105: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8303
Instructions for updating:
Use fn_output_signature instead
W0426 12:21:23.068551 140033449260608 deprecation.py:541] From /home/ovenoboyo/.local/lib/python3.8/site-packages/tensorflow/python/util/deprecation.py:616: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead
2022-04-26 12:21:59.950812: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsy

2022-04-26 12:21:59.968417: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 2257584128 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 1124401152/6230376448
2022-04-26 12:21:59.968455: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                      3246391296
InUse:                      2755000792
MaxInUse:                   3464741584
NumAllocs:                        7935
MaxAllocSize:               2743140352
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-04-26 12:21:59.968508: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-04-26 12:21:59.968522: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocat

2022-04-26 12:22:00.014512: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 2257584128 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 1023279104/6230376448
2022-04-26 12:22:00.014530: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                      3246391296
InUse:                      2869281704
MaxInUse:                   3464741584
NumAllocs:                        7946
MaxAllocSize:               2743140352
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-04-26 12:22:00.014574: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-04-26 12:22:00.014580: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocat

2022-04-26 12:22:00.185409: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 2372141056 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 687734784/6230376448
2022-04-26 12:22:00.185432: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                      3246391296
InUse:                      3177034664
MaxInUse:                   3464741584
NumAllocs:                        7994
MaxAllocSize:               2743140352
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-04-26 12:22:00.185474: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-04-26 12:22:00.185480: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocato

...

2022-04-26 12:22:01.525688: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 2300837888 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 1125515264/6230376448
2022-04-26 12:22:01.525713: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                      3246391296
InUse:                      2115390376
MaxInUse:                   3631679400
NumAllocs:                        8736
MaxAllocSize:               2743140352
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-04-26 12:22:01.525750: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-04-26 12:22:01.525756: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocat

2022-04-26 12:22:01.740349: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 2743140352 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 1125515264/6230376448
2022-04-26 12:22:01.740379: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                      3246391296
InUse:                      1725297720
MaxInUse:                   3631679400
NumAllocs:                        8895
MaxAllocSize:               2743140352
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-04-26 12:22:01.740418: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-04-26 12:22:01.740425: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocat

2022-04-26 12:22:01.788399: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 2302672896 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 1125515264/6230376448
2022-04-26 12:22:01.788426: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                      3246391296
InUse:                      1838871464
MaxInUse:                   3631679400
NumAllocs:                        8898
MaxAllocSize:               2743140352
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-04-26 12:22:01.788464: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-04-26 12:22:01.788470: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocat

2022-04-26 12:22:02.174103: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:288] gpu_async_0 cuMemAllocAsync failed to allocate 2334195712 bytes: CUDA error: out of memory (CUDA_ERROR_OUT_OF_MEMORY)
 Reported by CUDA: Free memory/Total memory: 1125515264/6230376448
2022-04-26 12:22:02.174129: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:293] Stats: Limit:                      3246391296
InUse:                      1743759784
MaxInUse:                   3653282216
NumAllocs:                        8962
MaxAllocSize:               2743140352
Reserved:                            0
PeakReserved:                        0
LargestFreeBlock:                    0

2022-04-26 12:22:02.174168: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocator.cc:56] Histogram of current allocation: (allocation_size_in_bytes, nb_allocation_of_that_sizes), ...;
2022-04-26 12:22:02.174174: E tensorflow/core/common_runtime/gpu/gpu_cudamallocasync_allocat

INFO:tensorflow:Step 25100 per-step time 1.269s
I0426 12:23:30.207896 140041548842816 model_lib_v2.py:705] Step 25100 per-step time 1.269s
INFO:tensorflow:{'Loss/classification_loss': 0.5190864,
 'Loss/localization_loss': 0.39292943,
 'Loss/regularization_loss': 0.1180743,
 'Loss/total_loss': 1.0300901,
 'learning_rate': 0.039990053}
I0426 12:23:30.212832 140041548842816 model_lib_v2.py:708] {'Loss/classification_loss': 0.5190864,
 'Loss/localization_loss': 0.39292943,
 'Loss/regularization_loss': 0.1180743,
 'Loss/total_loss': 1.0300901,
 'learning_rate': 0.039990053}


In [5]:
# for import error: cannot import name '_registerMatType' from 'cv2.cv2'
# !pip uninstall opencv-python-headless

In [6]:
# !pip install opencv-python-headless==4.1.2.30

In [7]:
pwd

'/home/ovenoboyo/projects/pbl/training'

saving the trained model after exporting it to saved_model dir

In [8]:
!python /home/ovenoboyo/projects/pbl/training/exporter_main_v2.py --input_type image_tensor --pipeline_config_path /home/ovenoboyo/projects/pbl/training/models/my_ssdresnet_101/pipeline.config --trained_checkpoint_dir /home/ovenoboyo/projects/pbl/training/models/my_ssdresnet_101 --output_directory /home/ovenoboyo/projects/pbl/training/saved_model

2022-04-26 09:23:08.787740: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 09:23:08.836171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 09:23:08.836380: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 09:23:08.841417: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

# viewing the obj dec

In [9]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-04-26 09:23:40.978730: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 09:23:41.006421: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 09:23:41.006617: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [10]:
%matplotlib inline

In [11]:
"""
Object Detection (On Image) From TF2 Saved Model
=====================================
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf
import cv2
import argparse
import cv2
import matplotlib.pyplot as plt

# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# PROVIDE PATH TO IMAGE DIRECTORY
IMAGE_PATHS = '/home/ovenoboyo/projects/pbl/training/images/test/fork_1.png'


# PROVIDE PATH TO MODEL DIRECTORY
PATH_TO_MODEL_DIR = '/home/ovenoboyo/projects/pbl/training/saved_model'

# PROVIDE PATH TO LABEL MAP
PATH_TO_LABELS = '/home/ovenoboyo/projects/pbl/training/annotations/label_map.pbtxt'

# PROVIDE THE MINIMUM CONFIDENCE THRESHOLD
MIN_CONF_THRESH = float(0.60)

# LOAD THE MODEL

import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"

print('Loading model...', end='')
start_time = time.time()

# LOAD SAVED MODEL AND BUILD DETECTION FUNCTION
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

# LOAD LABEL MAP DATA FOR PLOTTING

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.
    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.
    Args:
      path: the file path to the image
    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))




print('Running inference for {}... '.format(IMAGE_PATHS), end='')

image = cv2.imread(IMAGE_PATHS)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
input_tensor = tf.convert_to_tensor(image)
# The model expects a batch of images, so add an axis with `tf.newaxis`.
input_tensor = input_tensor[tf.newaxis, ...]

# input_tensor = np.expand_dims(image_np, 0)
detections = detect_fn(input_tensor)

# All outputs are batches tensors.
# Convert to numpy arrays, and take index [0] to remove the batch dimension.
# We're only interested in the first num_detections.
num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
               for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

image_with_detections = image.copy()

# SET MIN_SCORE_THRESH BASED ON YOU MINIMUM THRESHOLD FOR DETECTIONS
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_with_detections,
      detections['detection_boxes'],
      detections['detection_classes'],
      detections['detection_scores'],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.5,
      agnostic_mode=False)

print(image_with_detections)
# DISPLAYS OUTPUT IMAGE
# plt.imshow(image_with_detections)
# plt.show()

from PIL import Image
im = Image.fromarray(image_with_detections)
im.save("result.png")

# with open('result.png', 'a') as fp:
#     fp.write(image_with_detections)
# CLOSES WINDOW ONCE KEY IS PRESSED


Loading model...

2022-04-26 09:23:41.908256: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-26 09:23:41.908862: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 09:23:41.909064: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-26 09:23:41.909213: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Done! Took 7.483151197433472 seconds
Running inference for /home/ovenoboyo/projects/pbl/training/images/test/fork_1.png... 

2022-04-26 09:23:50.884633: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8303


[[[255 255 249]
  [251 253 254]
  [250 254 255]
  ...
  [127 255   0]
  [127 255   0]
  [127 255   0]]

 [[255 253 252]
  [255 254 255]
  [127 255   0]
  ...
  [127 255   0]
  [127 255   0]
  [127 255   0]]

 [[255 253 255]
  [255 253 255]
  [127 255   0]
  ...
  [127 255   0]
  [127 255   0]
  [127 255   0]]

 ...

 [[255 255 255]
  [255 255 255]
  [127 255   0]
  ...
  [127 255   0]
  [127 255   0]
  [127 255   0]]

 [[255 255 255]
  [255 255 255]
  [127 255   0]
  ...
  [127 255   0]
  [127 255   0]
  [127 255   0]]

 [[255 255 255]
  [255 255 255]
  [127 255   0]
  ...
  [127 255   0]
  [127 255   0]
  [127 255   0]]]


In [12]:
pwd

'/home/ovenoboyo/projects/pbl/training'